Voici **la procédure complète, propre et technique** pour utiliser **un Service Account** avec Google Calendar (la méthode PRO : stable, sans OAuth, sans test users, sans refresh tokens).

Je te propose un workflow en **3 étapes** :
1️⃣ Créer le service account
2️⃣ L’autoriser sur ton agenda Google
3️⃣ L’utiliser dans ton code pour lire/écrire des événements

---

# ✅ 1. Création du Service Account (Google Cloud)

1. Va sur :
   [https://console.cloud.google.com/iam-admin/serviceaccounts](https://console.cloud.google.com/iam-admin/serviceaccounts)
2. Sélectionne ton projet
3. Clique **Create Service Account**

**Remplir :**

* Service account name : `calendar-bot` (par ex.)
* ID généré automatiquement

👉 Clique **Create and Continue**

4. Ne donne *aucun rôle* (Google Calendar ne se gère pas via IAM mais via partage d’agenda).
   👉 Clique **Done**

5. Dans la liste, clique sur ton service account nouvellement créé.

6. Onglet **Keys** → **Add Key** → **Create new key**

* Type : **JSON**
* Enregistre le fichier → c’est **ton fichier de credentials** (ex. `service.json`)

⚠️ Ne le partage jamais.

---

# ✅ 2. Autoriser ce service account à accéder à TON agenda

Ton service account possède une adresse type :

```
calendar-bot@ton-projet.iam.gserviceaccount.com
```

➡️ Ce n’est pas un “vrai” compte Google, mais on peut **partager un agenda** avec lui.

---

### 📌 Sur Google Calendar :

1. Ouvre Google Calendar
2. Dans la colonne de gauche → **Mes agendas**
3. Clique sur les trois points → **Paramètres et partage**
4. Descends jusqu’à : **Partager avec des personnes spécifiques**
5. Ajoute :

```
calendar-bot@ton-projet.iam.gserviceaccount.com
```

6. Donne lui les droits :

```
Apporter des modifications et gérer le partage
```

🎯 Sans ce partage → ton service account n’a AUCUN accès → 403 garanti.

---

# ✅ 3. Utiliser le service account dans ton code

Ton code devient extrêmement simple :

```python
from google.oauth2 import service_account
from googleapiclient.discovery import build

SCOPES = ["https://www.googleapis.com/auth/calendar"]
SERVICE_ACCOUNT_FILE = "service.json"  # le fichier téléchargé

# Création du credential
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=SCOPES,
)

# Construire le client Calendar
service = build("calendar", "v3", credentials=creds)

# ID de l'agenda — "primary" fonctionne si tu as partagé TON agenda perso
calendar_id = "primary"

# Test : récupérer les 10 prochains événements
events_result = service.events().list(
    calendarId=calendar_id,
    maxResults=10,
    singleEvents=True,
    orderBy="startTime",
).execute()

events = events_result.get("items", [])

for event in events:
    print(event["summary"], event["start"])
```

🎉 Aucune page OAuth
🎉 Aucun token.json
🎉 Aucun refresh
🎉 Aucune validation Google nécessaire
🎉 Ultrastable pour backend (Django, script, cron, etc.)

---

# 🔍 4. Vérifier qu’un créneau est disponible

Exemple : vérifier si un créneau `[start, end]` est libre :

```python
import datetime

start = datetime.datetime(2025, 1, 1, 14, 0).isoformat() + "Z"
end   = datetime.datetime(2025, 1, 1, 15, 0).isoformat() + "Z"

events = service.events().list(
    calendarId="primary",
    timeMin=start,
    timeMax=end,
    singleEvents=True,
    orderBy="startTime",
).execute().get("items", [])

if len(events) == 0:
    print("Créneau dispo !")
else:
    print("Créneau occupé :", events)
```

---

# 🧱 5. Ajouter un événement (ex : RDV client)

```python
event = {
    "summary": "Rendez-vous client",
    "start": {"dateTime": start, "timeZone": "Europe/Paris"},
    "end":   {"dateTime": end,   "timeZone": "Europe/Paris"},
}

event_created = service.events().insert(
    calendarId="primary",
    body=event
).execute()

print("Event created:", event_created.get("htmlLink"))
```


In [2]:
from reservations.utils import *

In [3]:
# ID de l'agenda
calendar_id_resa = "0047459c732e33256d4e298bd3d24e20c3d1f80fce13ef7c8eb5f3d91ed8b31f@group.calendar.google.com"
calendar_id_perso = "virgil.mesle@gmail.com"

Workflow : creer un agenda "reservation" où on renseigne les créneaux voulus

In [4]:

d_start = datetime(2025, 11, 17, 15, 29, tzinfo=zoneinfo.ZoneInfo("Europe/Paris"))
d_end = d_start + timedelta(hours=1)
# get_events_current_week(calendar_id_resa)


In [5]:
get_week_date_range()

(datetime.datetime(2025, 11, 17, 0, 0, tzinfo=<DstTzInfo 'Europe/Paris' CET+1:00:00 STD>),
 datetime.datetime(2025, 11, 23, 23, 59, 59, tzinfo=<DstTzInfo 'Europe/Paris' CET+1:00:00 STD>),
 '2025-11-17T00:00:00+01:00',
 '2025-11-23T23:59:59+01:00')

In [6]:
get_events_current_week(calendar_id_resa)

['20/11/2025 - 10:00 à 18:00',
 '21/11/2025 - 10:00 à 18:00',
 '22/11/2025 - 10:00 à 18:00',
 '23/11/2025 - 10:00 à 18:00']